# Agency Prefix Code Profiles
Which agencies apply the most or have the most prefix codes? Which ones have the least? 

Data: 
* [Oblgiated list](https://dot.ca.gov/programs/local-assistance/reports/e-76-obligated)

* running notebook with updated locode data 'obligated_list_1202.csv'

In [1]:
import numpy as np
import pandas as pd
from siuba import *
import _clean_data
import altair as alt
import altair_saver
from plotnine import *

from shared_utils import altair_utils
alt.themes.enable("fivethirtyeight")
from calitp import *

import ipywidgets as widgets
from ipywidgets import *
from IPython.display import Markdown
from IPython.core.display import display

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [2]:
df = _clean_data.read_data()
df = _clean_data.clean_data(df)
df = _clean_data.prefix_cleaning(df)
df = _clean_data.clean_agency_names(df)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/home/jovyan/data-analyses/dla/e76_obligated_funds/_clean_data.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,ac_requested,total_requested,status_comment,locode,dist,status,dist_processing_days,hq_processing_days,fhwa_processing_days,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
0,Obligated,BPMP,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,0.0,0.00,Authorized,5904,1,E-76 approved on,NaN,0.0,9.0,HBPLOCAL,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaT,NaT,NON-MPO,5904,2018.0,Humboldt County
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,0.0,13000.00,Authorized,5910,1,E-76 approved on,1.0,1.0,7.0,NaN,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,32D0,2018.0,Mendocino County
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,0.0,51394.58,Authorized,5904,1,E-76 approved on,14.0,0.0,6.0,NaN,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,4820,2018.0,Humboldt County
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,0.0,247002.00,Authorized,5924,3,E-76 approved on,4.0,16.0,0.0,SAC25086,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,5924,2018.0,Sacramento County
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,5680921.0,5702041.00,Authorized,5924,3,E-76 approved on,7.0,16.0,0.0,SAC24753,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,5924,2018.0,Sacramento County


In [4]:
#making sure the locode and agency updated in the data.
## if updated, there will only be one agency listed.
## if multiple agencies listed, go back to add_locode.ipynb and run that notebook

df>>filter(_.locode=='6000')>>count(_.primary_agency_name)

,primary_agency_name,n


## How many unique prefix codes per agency

### Top Agencies
finding the top agencies with the most unique prefix codes

In [5]:
df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(-_.n)>>filter(_.n>=15)

,primary_agency_name,n
276,Los Angeles,34
277,Los Angeles County,31
434,San Bernardino Associated Governments,29
71,Caltrans,27
534,Stockton,23
416,Riverside County,22
466,San Luis Obispo County,22
149,El Dorado County,21
442,San Diego Association of Governments,20
215,Imperial County,19


In [6]:
top = df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(-_.n)>>filter(_.n>=20)

In [7]:
top

,primary_agency_name,n
276,Los Angeles,34
277,Los Angeles County,31
434,San Bernardino Associated Governments,29
71,Caltrans,27
534,Stockton,23
416,Riverside County,22
466,San Luis Obispo County,22
149,El Dorado County,21
442,San Diego Association of Governments,20


In [8]:
@interact
def find_agencies_top(agency=top.primary_agency_name.unique().tolist()):

            count_top = (df>>filter(_.primary_agency_name==agency)>> count(_.prefix)>>arrange(-_.n))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_top)


interactive(children=(Dropdown(description='agency', options=('Los Angeles', 'Los Angeles County', 'San Bernar…

#### Printing top 5 programs for the top agencies


In [9]:
top

,primary_agency_name,n
276,Los Angeles,34
277,Los Angeles County,31
434,San Bernardino Associated Governments,29
71,Caltrans,27
534,Stockton,23
416,Riverside County,22
466,San Luis Obispo County,22
149,El Dorado County,21
442,San Diego Association of Governments,20


In [10]:
def find_agencies_top(agency=top.primary_agency_name.unique().tolist()):
    
    for agency in top.primary_agency_name.unique().tolist(): 
            count_top = ((df>>filter(_.primary_agency_name==agency)>> count(_.prefix)>>arrange(-_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_top)
            
    return

In [11]:
agency=top.primary_agency_name.unique().tolist()
find_agencies_top(agency)

**Top Prefix Codes Los Angeles uses:**

,prefix
26,STPL
20,HSIPL
10,CML
2,ATPL
5,BHLS


**Top Prefix Codes Los Angeles County uses:**

,prefix
12,ER
18,HSIPL
1,ACSTP
4,BPMPL
23,STPL


**Top Prefix Codes San Bernardino Associated Governments uses:**

...

,prefix
8,BRLO
17,HSIPL
5,BHLS
9,BRLS
4,BHLO


**Top Prefix Codes El Dorado County uses:**

,prefix
6,BRLO
8,CML
15,HSIPL
12,ER
20,STPL


**Top Prefix Codes San Diego Association of Governments uses:**

,prefix
13,RPSTPL
19,STPL
3,CMLG
2,CML
8,FTASTPL


Popular codes in top agencies:
   * STP/STLN: Surface Transportation Program/Surface Transportation Program Local National Highway System
   * HSIP: Highway Safety Improvement Program 
   * CML: Congestion Mitigation & Air Quality Improvement Program 
   * BHLS: ?
   * BRLO: Highway Bridge Program (HBP) - Off Federal-aid System 

### Bottom Agencies

In [12]:
(df 
     >> group_by(_.primary_agency_name) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=1)
 
)

,primary_agency_name,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
13,Amador City,1
15,Amador County Transportation Commission,1
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


In [13]:
df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

,primary_agency_name,n
4,Alameda Corridor Transportation Authority,1
8,Alameda County Transportation Improvement Auth...,1
9,Alameda County Waste Management Authority,1
13,Amador City,1
15,Amador County Transportation Commission,1
...,...,...
587,Walnut Creek,3
592,West Covina,3
597,Westminster,3
605,Winters,3


In [14]:
bottom = df >> group_by(_.primary_agency_name) >> summarize(n=_.prefix.nunique()) >> arrange(_.n)>>filter(_.n<=3)

In [15]:
bottom.sample(9)

,primary_agency_name,n
592,West Covina,3
117,Cotati,2
43,Bellflower,3
573,University of California - Davis,1
9,Alameda County Waste Management Authority,1
94,Clayton,1
596,Westlake Village,2
56,Buena Park,1
48,Biggs,2


In [16]:
agency=bottom.primary_agency_name.unique().tolist()


def find_agencies_bottom(agency=bottom.primary_agency_name.unique().tolist()):
    
    for agency in bottom.primary_agency_name.unique().tolist(): 
            count_bottom = ((df>>filter(_.primary_agency_name==agency)>> count(_.prefix)>>arrange(_.n)).head(5)>>select(_.prefix))
            
            display(Markdown(f"**Top Prefix Codes {agency} uses:**"))
            display(count_bottom)
            
    return

In [17]:
print(len(agency))

302


In [18]:
find_agencies_bottom(agency)

**Top Prefix Codes Alameda Corridor Transportation Authority uses:**

,prefix
0,HPLUL


**Top Prefix Codes Alameda County Transportation Improvement Authority uses:**

,prefix
0,STPL


**Top Prefix Codes Alameda County Waste Management Authority uses:**

...

,prefix
2,STPL
0,ATPL
1,HSIPL


**Top Prefix Codes Winters uses:**

,prefix
1,HPLUL
2,STPL
0,BRLS


**Top Prefix Codes Yreka City uses:**

,prefix
2,BRLS
0,ACSTP
1,BPMP


Popular codes in bottom agencies: 
   * FTACML
   * STP/RPSTP
   * HSIP
   * CML 
   * BRLS
   * BRLO
   * CMLNI
    

* 140 agencies with 1 unique prefix code.
* 339 agencies with 3 or less prefix codes. 

## In which Caltrans Districts do these agencies have the most obligations?

### For the agencies in districts that applied the least

* joining the bottom agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [19]:
bottom = (df 
     >> group_by(_.primary_agency_name) 
     >> summarize(n=_.prefix.nunique())
     >> filter(_.n<=3)
)

In [20]:
bottom_join = inner_join(df, bottom, on = "primary_agency_name")

In [21]:
bottom_dist = bottom_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [22]:
bottom_dist.rename(columns={'n': 'bottom_n'}, inplace=True)

In [23]:
bottom_dist

,dist,bottom_n
3,4,394
6,7,382
9,10,352
5,6,264
2,3,259
7,8,209
10,11,129
11,12,114
4,5,81
0,1,60


### For the agencies in districts that applied the most

* joining the top agencies to the full df
* grouping by dsitrict to count the number of obligation occurances 

In [24]:
top_join = inner_join(df, top, on = "primary_agency_name")

In [25]:
top_dist = top_join >> group_by(_.dist) >> count(_.dist) >> arrange(-_.n)

In [26]:
top_dist.rename(columns={'n': 'top_n'}, inplace=True)

In [27]:
top_dist

,dist,top_n
7,7,959
3,3,316
10,10,262
8,8,219
5,5,133
11,11,131
12,12,13
4,4,11
0,0,4
6,6,3


### Number of Obligations for Top and Bottom on District
 
* joining the dataframes we just made to compare the top and bottom easier
* finding which districts have the most obligations 

In [28]:
dist = full_join(bottom_dist, top_dist, on = "dist")

In [29]:
dist>>arrange(_.dist)

,dist,bottom_n,top_n
12,0,NaN,4
9,1,60.0,2
10,2,52.0,1
4,3,259.0,316
0,4,394.0,11
8,5,81.0,133
3,6,264.0,3
1,7,382.0,959
5,8,209.0,219
11,9,39.0,1


In [30]:
dist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 12
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dist      13 non-null     int64  
 1   bottom_n  12 non-null     float64
 2   top_n     13 non-null     int64  
dtypes: float64(1), int64(2)
memory usage: 416.0 bytes


In [31]:
dist['bottom_n'] = dist['bottom_n'].fillna(0)

In [32]:
dist['bottom_n'] = dist['bottom_n'].astype(int)

In [33]:
dist.rename(columns={'dist': 'district'}, inplace=True)

In [34]:
dist>>arrange(_.district)

,district,bottom_n,top_n
12,0,0,4
9,1,60,2
10,2,52,1
4,3,259,316
0,4,394,11
8,5,81,133
3,6,264,3
1,7,382,959
5,8,209,219
11,9,39,1


## Findings:



* **339 agencies** have **3 or less** unique prefixes. 
* **80 agencies** have **10 or more** unique prefixes, with the following **9 agenceies** listed with **20 or more**. 
* **District 7 and District 2** have the most obligations out of the top and bottom agencies 



| Top Agencies  | Bottom Agencies (sample of 9)|
| ------------- | ------------- | 
| Los Angeles | Sonoma County Transportation Authority | 
| Caltrans  |Port Of Long Beach |   
| Los Angeles County |Newport Beach |    
| San Bernardino Associated Governments | Portola | 
| San Diego | Calaveras Council Of Governments | 
| Riverside County | Hawthorne | 
| San Luis Obispo County | Corcoran |  
| Stockton | Claremont | 
| El Dorado County | El Cajon |  


